In [2]:
#connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#chargement des packages
import pandas as pd
import numpy as np

In [4]:
#import du fichier via ggdrive
#Lien perso ggdrive
file_path="/content/drive/MyDrive/projet_python/df_dating.csv"
df=pd.read_csv(file_path,sep=',')

##Récupération des composantes de l'AFDM

In [22]:
#Récupération des variables quantitatives
#df_quanti=df[['age','income','attr1_1', 'sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1','age_o','pf_o_att','pf_o_sin','pf_o_int','pf_o_fun','pf_o_amb','pf_o_sha']]
df_quanti=df[['age','attr1_1', 'intel1_1','age_o','pf_o_sin']]

#On centre et on réduit les variables quantitatives (même échelle)
ZQ = (df_quanti.values-np.mean(df_quanti.values,axis=0))/np.std(df_quanti.values,axis=0) 

In [69]:
#Récupération des variables qualitatives codées : centrer et réduire
#df_quali_quanti=df[['round','gender','condtn','position','order','wave','samerace','dec_o','exphappy','expnum','attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','imprelig','imprace','yoga','sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping']]
df_quali_quanti=df[['condtn','wave','exphappy','fun_o','gaming']]

#On centre et on réduit les variables quantitatives (même échelle)
ZQQ = (df_quali_quanti.values-np.mean(df_quali_quanti.values,axis=0))/np.std(df_quali_quanti.values,axis=0) 

In [62]:
#Récupération des variables qualitatives
#df_qualitatives=df[['race_o','field','from','career','zipcode','goal','date','go_out']]
df_quali=df[['go_out','date']]
#codage 0/1 
CQ = pd.get_dummies(df_quali) 
CQd=CQ.drop(columns=['go_out_Missing value','date_Missing value'])

In [63]:
#proportions 
p_k = np.mean(CQd.values,axis=0) 

#standardisation des indicatrices 
ZD = CQd.values/np.sqrt(p_k) 

In [64]:
#assembler tous les blocs
Z = np.concatenate((ZQ,ZQQ,ZD),axis=1) 

In [16]:
pip install fanalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.3 MB/s 


In [65]:
import fanalysis
from fanalysis.pca import PCA 
#instanciation - les données sont déjà standardisées 
afdm = PCA(std_unit=False,row_labels=df.index,col_labels=[df_quanti.columns,df_quali_quanti.columns,CQ.columns]) 
#lancement des calculs 
afdm.fit(Z)

PCA(col_labels=[Index(['age', 'attr1_1', 'intel1_1', 'age_o', 'pf_o_sin'], dtype='object'),
                Index(['condtn', 'wave', 'exphappy', 'fun_o', 'gaming'], dtype='object'),
                Index(['go_out_1.0', 'go_out_2.0', 'go_out_3.0', 'go_out_4.0', 'go_out_5.0',
       'go_out_6.0', 'go_out_7.0', 'go_out_Missing value', 'date_1.0',
       'date_2.0', 'date_3.0', 'date_4.0', 'date_5.0', 'date_6.0', 'date_7.0',
       'date_Missing value'],
      dtype='object')],
    row_labels=RangeIndex(start=0, stop=6804, step=1), std_unit=False)

In [66]:
#affichage des valeurs propres 
valpropres=afdm.eig_[0]
n=len(valpropres)
print(n)

24


##Création de l'échantillon d'apprentissage et de l'échantillon test

In [70]:
#importer la librairie pour l'apprentissage automatique et la modélisation statistique
import sklearn
from sklearn.model_selection import train_test_split
def modele(df_explicatives,train_size=0.7) : 
  #variables explicatives
  X=df_explicatives
  #variable cible
  Y=df.match
  #distribution des classes
  Y.value_counts(normalize=True)
  #échantillon d'apprentissage (70%) et échantillon test (30%)
  XTrain,XTest,YTrain,YTest=train_test_split(X,Y,train_size=0.7,random_state=0,stratify=df.match)
  return(XTrain,XTest,YTrain,YTest)

##Arbre de decision

In [101]:
df_exp0=df_quanti.join(df_quali_quanti)

       age  attr1_1  intel1_1  age_o  pf_o_sin  condtn  wave  exphappy  \
0     27.0     30.0      30.0   29.0      10.0       2    17       6.0   
1     27.0     30.0      30.0   23.0      15.0       2    17       6.0   
2     27.0     30.0      30.0   23.0      15.0       2    17       6.0   
3     27.0     30.0      30.0   23.0       8.0       2    17       6.0   
4     27.0     30.0      30.0   22.0       5.0       2    17       6.0   
...    ...      ...       ...    ...       ...     ...   ...       ...   
6799  21.0     15.0      20.0   24.0      10.0       1     1       3.0   
6800  21.0     15.0      20.0   23.0       5.0       1     1       3.0   
6801  21.0     15.0      20.0   22.0      18.0       1     1       3.0   
6802  21.0     15.0      20.0   22.0       0.0       1     1       3.0   
6803  21.0     15.0      20.0   27.0      20.0       1     1       3.0   

          fun_o  gaming  
0      3.000000     5.0  
1      3.000000     5.0  
2      6.000000     5.0  
3      

In [88]:
df_exp1=df_exp0.join(CQd)

#Sans les variables qualitatives

In [94]:
XTrain,XTest,YTrain,YTest=modele(df_exp0)

In [95]:
#Vérification (pourcentage apprentissage, test)
print(XTrain.shape)
print(XTest.shape)

#Vérification échantillonage stratifié 
train_match=YTrain.value_counts()
freq_train_match = train_match / train_match.sum()
print(freq_train_match)

test_match=YTest.value_counts()
freq_test_match = test_match / test_match.sum()
print(freq_test_match)

(4762, 10)
(2042, 10)
0    0.831583
1    0.168417
Name: match, dtype: float64
0    0.831538
1    0.168462
Name: match, dtype: float64


Modèle

In [116]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
#instanciation de l'arbre de décision avec les paramètres par défaut
arbre=DecisionTreeClassifier(min_samples_split=20,min_samples_leaf=10,max_depth=None,random_state=0)
#modelisation sur les données d'apprentissage avec la méthode de l'arbre de décision (paramètres par défaut)
arbre.fit(XTrain,YTrain)
#affichage graphique de l'arbre
from sklearn.tree import plot_tree
#affichage plus grand
#plt.figure(figsize=(20,20))
#plot_tree(arbre,feature_names=XTrain.columns,filled=True)
#plt.show()

Prédictions

In [117]:
#importance des variables
pd.DataFrame(arbre.feature_importances_,index=XTrain.columns,columns=['Importance'])
#prédiction sur l'échantillon test avec le modèle de l'arbre (paramètres par défaut)
predic=arbre.predict(XTest)

Mesures des performances du modèle

In [118]:
#comptage des prédictions
pd.Series(predic).value_counts()
#matrice de confusion associée au modèle de l'arbre (paramètres par défaut de l'algorithme)
from sklearn import metrics
confusion = metrics.confusion_matrix(y_true=YTest, y_pred=predic)
print(f"Matrice de confusion : {confusion}")
#calcul de l'erreur dans ce cas : modèle de l'arbre, paramètres par défaut
erreur=1-metrics.accuracy_score(YTest,predic)
print(f"Taux d'erreur : {erreur}")
#calcul du F1_score
F1_score=metrics.f1_score(YTest, y_pred=predic,pos_label=1, average='macro')
print(f"F1_score : {F1_score}")

Matrice de confusion : [[1565  133]
 [ 277   67]]
Taux d'erreur : 0.20078354554358468
F1_score : 0.5652521601861084


#Avec les variables qualitatives

In [114]:
XTrain,XTest,YTrain,YTest=modele(df_exp1)
#instanciation de l'arbre de décision avec les paramètres par défaut
arbre=DecisionTreeClassifier(min_samples_split=20,min_samples_leaf=10,max_depth=None,random_state=0)
#modelisation sur les données d'apprentissage avec la méthode de l'arbre de décision (paramètres par défaut)
arbre.fit(XTrain,YTrain)
#prédiction sur l'échantillon test avec le modèle de l'arbre (paramètres par défaut)
predic=arbre.predict(XTest)
#calcu du F1_score
F1_score=metrics.f1_score(YTest, y_pred=predic,pos_label=1, average='macro')
print(f"F1_score : {F1_score}")

F1_score : 0.5652521601861084


Les F1_score avec les variables qualitatives est moins élevé que sans les variables qualitatives

#Avec les composantes principales issues de l'AFDM

In [111]:
df_exp2=pd.DataFrame(afdm.row_coord_[:,:8],columns=['F1','F2','F3','F4','F5','F6','F7','F8'])

In [119]:
XTrain,XTest,YTrain,YTest=modele(df_exp2)
#instanciation de l'arbre de décision avec les paramètres par défaut
arbre=DecisionTreeClassifier(min_samples_split=20,min_samples_leaf=10,max_depth=None,random_state=0)
#modelisation sur les données d'apprentissage avec la méthode de l'arbre de décision (paramètres par défaut)
arbre.fit(XTrain,YTrain)
#prédiction sur l'échantillon test avec le modèle de l'arbre (paramètres par défaut)
predic=arbre.predict(XTest)
#calcu du F1_score
F1_score=metrics.f1_score(YTest, y_pred=predic,pos_label=1, average='macro')
print(f"F1_score : {F1_score}")

F1_score : 0.5385237119103262


Le F1_score est moins élevé